In [1]:
import pandas as pd
from pathlib import Path
import json
import re
from calendar import month_name

from coffeaphylogeo.definitions import Definitions

from typing import List

In [2]:
defs = Definitions()
climate_dir = defs.get_geoclim_path("climate_data")
enviro_dir = defs.get_geoclim_path("environment_data")

In [3]:
defs.geoclim_files

{'raw_table': 'coff_madag_species_summary.xlsx',
 'gps_all': 'species_locations_all_positions.csv',
 'gbs_filtered': 'species_locations_gbs_only.csv',
 'madaclim_current': 'madaclim_current.tif',
 'madaclim_enviro': 'madaclim_enviro.tif',
 'clim_metadata': 'clim_metadata.json',
 'env_metadata': 'enviro_metadata.json',
 'clim_data_format': 'clim_data_format.json',
 'env_data_format': 'env_data_format.json'}

In [4]:
with open(climate_dir / defs.geoclim_files["clim_data_format"], "r") as f:
    clim_format = json.load(f)

In [5]:
df_clim = pd.read_json(clim_format["table_0"])
df_clim["data_type"] = "clim"

In [6]:
df_clim.head()

,Layers,Climate variable,data_type
0,1-12,Monthly minimum temperature (°C x 10),clim
1,13-24,Monthly maximum temperature (°C x 10),clim
2,25-36,Monthly total precipitation (mm.month-1),clim
3,37-55,Bioclimatic variables (bioclim),clim
4,56-67,Monthly potential evapotranspiration (mm.month-1),clim


In [7]:
def split_layers(row, layers_col_name: str):
    """
    Split "Layers" column and create new rows

    Args:
        row (pd.Series): Row of data from a DataFrame
        layers_col_name (str): Name of the column to split

    Returns:
        pd.DataFrame: DataFrame with split rows

    Examples:
        >>> df = pd.DataFrame({"A": ["x1-3", "y4"], "B": ["foo", "bar"]})
        >>> result = df.apply(split_repeating_vars, axis=1, col_to_split="A", col_to_keep="B")
        >>> df_result = pd.concat(result.tolist(), axis=0).reset_index(drop=True)
        >>> print(df_result)
             A    B
        0   x1  foo
        1   x2  foo
        2   x3  foo
        3   y4  bar
    """
    if "-" in row[layers_col_name]:
        start, end = map(int, row[layers_col_name].split("-"))
        index = range(start, end+1)
    else:
        index = [int(row[layers_col_name])]
    
    data = {
        "Climate variable": [row["Climate variable"]] * len(index),
        "data_type": [row["data_type"]] * len(index)
    }
    
    return pd.DataFrame(data, index=index)

In [8]:
df_clim = pd.concat((df_clim.apply(split_layers, args=("Layers", ), axis=1)).to_list()).reset_index()

In [9]:
df_clim.columns = ["layer_number", "geoclim_feature", "geoclim_type"]

In [10]:
df_clim

,layer_number,geoclim_feature,geoclim_type
0,1,Monthly minimum temperature (°C x 10),clim
1,2,Monthly minimum temperature (°C x 10),clim
2,3,Monthly minimum temperature (°C x 10),clim
3,4,Monthly minimum temperature (°C x 10),clim
4,5,Monthly minimum temperature (°C x 10),clim
...,...,...,...
65,66,Monthly potential evapotranspiration (mm.month-1),clim
66,67,Monthly potential evapotranspiration (mm.month-1),clim
67,68,Annual potential evapotranspiration (mm),clim
68,69,Annual climatic water deficit (mm),clim


Monthly features bio

In [11]:
with open(climate_dir /  defs.geoclim_files["clim_metadata"],"r") as f:
    clim_meta = json.load(f)

In [12]:
[key for key in clim_meta]

['table_0', 'table_1', 'table_2', 'table_3']

In [13]:
bio_monthly_feats = pd.read_json(clim_meta["table_0"])

In [14]:
bio_monthly_feats.columns = ["layer_name", "layer_description"]

In [15]:
bio_monthly_feats

,layer_name,layer_description
0,tmin1-12,Monthly minimum temperature (°C x 10)
1,tmax1-12,Monthly maximum temperature (°C x 10)
2,prec1-12,Monthly total precipitation (mm.month-1)


In [16]:
def split_repeating_vars(row: pd.Series, col_to_split: str, col_to_keep: str) -> pd.DataFrame:
    """Split a column containing repeating values into separate rows.

    This function takes a row of data from a DataFrame and splits the value in the column specified by `col_to_split` if it contains a hyphen. The function returns a new DataFrame with rows for each value in the specified range and with values from the column specified by `col_to_keep`. The month name is appended to the `col_to_keep` values.

    Args:
        row (pd.Series): A row of data from a DataFrame.
        col_to_split (str): The name of the column to split.
        col_to_keep (str): The name of the column to keep.

    Returns:
        pd.DataFrame: A DataFrame with split rows.
    """
    
    # Extract the range and layername to a new smaller df of len(range(start, end))
    if "-" in row[col_to_split]:
        start = int(re.search("\d+", row[col_to_split].split("-")[0]).group())
        end = int(row[col_to_split].split("-")[1])
        name = re.search("[a-z]*", row[col_to_split]).group()
                
        # Create a DataFrame with the split values and the description column
        df = pd.DataFrame({col_to_split: [f"{name}{month}" for month in range(start, end+1)],
                           col_to_keep: [f"{row[col_to_keep]} - {month_name[month]}" for month in range(start, end+1)]})
        return df
    
    # When no changes to changes to row
    else:
        df = pd.DataFrame({col_to_split: [row[col_to_split]], col_to_keep: [row[col_to_keep]]})
        return df

In [17]:
bio_monthly_feats = pd.concat(bio_monthly_feats.apply(split_repeating_vars, axis=1, args=("layer_name", "layer_description", )).to_list(), ignore_index=True)

In [18]:
bio_monthly_feats

,layer_name,layer_description
0,tmin1,Monthly minimum temperature (°C x 10) - January
1,tmin2,Monthly minimum temperature (°C x 10) - February
2,tmin3,Monthly minimum temperature (°C x 10) - March
3,tmin4,Monthly minimum temperature (°C x 10) - April
4,tmin5,Monthly minimum temperature (°C x 10) - May
5,tmin6,Monthly minimum temperature (°C x 10) - June
6,tmin7,Monthly minimum temperature (°C x 10) - July
7,tmin8,Monthly minimum temperature (°C x 10) - August
8,tmin9,Monthly minimum temperature (°C x 10) - September
9,tmin10,Monthly minimum temperature (°C x 10) - October


In [31]:
bio_monthly_feats[bio_monthly_feats["layer_name"].str.contains("prec")]["layer_description"].str.split(" - ").str[0].unique()[0]

'Monthly total precipitation (mm.month-1)'

In [38]:
def add_layer_numbers_bio_monthly(df: pd.DataFrame) -> pd.DataFrame:
    
    # Get the geoclim feature name and sub df for each layer_name categories
    monthly_features = {}
    categories_layer_name = df["layer_name"].str.extract("(^[a-zA-Z]+)")[0].unique()

    # Add layer number to bio_monthly df
    current_start_layer = 1
    for layer_name in categories_layer_name:
        
        # Extract common base layer name
        category_feature_val = df[df["layer_name"].str.contains(layer_name)]["layer_description"].str.split(" - ").str[0].unique()[0]
        monthly_features[f"{layer_name}_cat_feature"] = category_feature_val
        
        # Get df associated with common category
        category_feature_df = pd.DataFrame(df[df["layer_name"].str.contains(layer_name)])
        monthly_features[f"{layer_name}_df"] = category_feature_df

        # Assign layer number according to current category
        df.loc[df["layer_name"].str.contains(layer_name), "layer_number"] = range(current_start_layer, current_start_layer + len(category_feature_df))
        current_start_layer += len(category_feature_df)

    df["layer_number"] = df["layer_number"].astype(int)

    return df


In [41]:
bio_monthly_feats = add_layer_numbers_bio_monthly(bio_monthly_feats)

In [42]:
bio_monthly_feats

,layer_name,layer_description,layer_number
0,tmin1,Monthly minimum temperature (°C x 10) - January,1
1,tmin2,Monthly minimum temperature (°C x 10) - February,2
2,tmin3,Monthly minimum temperature (°C x 10) - March,3
3,tmin4,Monthly minimum temperature (°C x 10) - April,4
4,tmin5,Monthly minimum temperature (°C x 10) - May,5
5,tmin6,Monthly minimum temperature (°C x 10) - June,6
6,tmin7,Monthly minimum temperature (°C x 10) - July,7
7,tmin8,Monthly minimum temperature (°C x 10) - August,8
8,tmin9,Monthly minimum temperature (°C x 10) - September,9
9,tmin10,Monthly minimum temperature (°C x 10) - October,10


In [43]:
bioclim_feats = pd.read_json(clim_meta["table_1"])

In [44]:
bioclim_feats

,Bioclimatic variable,description
0,bio1,Annual mean temperature
1,bio2,Mean diurnal range (mean of monthly (max temp ...
2,bio3,Isothermality (BIO2/BIO7) (x 100)
3,bio4,Temperature seasonality (standard deviation x ...
4,bio5,Max temperature of warmest month
5,bio6,Min temperature of coldest month
6,bio7,Temperature annual range (BIO5-BIO6)
7,bio8,Mean temperature of wettest quarter
8,bio9,Mean temperature of driest quarter
9,bio10,Mean temperature of warmest quarter


In [45]:
bioclim_feats.columns = ["layer_name", "layer_description"]

In [46]:
bioclim_feats.head()

,layer_name,layer_description
0,bio1,Annual mean temperature
1,bio2,Mean diurnal range (mean of monthly (max temp ...
2,bio3,Isothermality (BIO2/BIO7) (x 100)
3,bio4,Temperature seasonality (standard deviation x ...
4,bio5,Max temperature of warmest month


In [47]:
current_start_layer = len(bio_monthly_feats) + 1
bioclim_feats["layer_number"] = range(current_start_layer, current_start_layer + len(bioclim_feats))
bioclim_feats

,layer_name,layer_description,layer_number
0,bio1,Annual mean temperature,37
1,bio2,Mean diurnal range (mean of monthly (max temp ...,38
2,bio3,Isothermality (BIO2/BIO7) (x 100),39
3,bio4,Temperature seasonality (standard deviation x ...,40
4,bio5,Max temperature of warmest month,41
5,bio6,Min temperature of coldest month,42
6,bio7,Temperature annual range (BIO5-BIO6),43
7,bio8,Mean temperature of wettest quarter,44
8,bio9,Mean temperature of driest quarter,45
9,bio10,Mean temperature of warmest quarter,46


In [48]:
evap_feats = pd.read_json(clim_meta["table_2"])
evap_feats

,Bioclimatic variable,Description
0,pet1-12,Monthly potential evapotranspiration from the ...
1,pet,Annual potential evapotranspiration from the T...


In [49]:
evap_feats.columns = ["layer_name", "layer_description"]

In [50]:
evap_feats = pd.concat(evap_feats.apply(split_repeating_vars, axis=1, args=("layer_name", "layer_description", )).to_list(), ignore_index=True)

In [51]:
current_start_layer = max(bioclim_feats["layer_number"]) + 1
evap_feats["layer_number"] = range(current_start_layer, current_start_layer + len(evap_feats))
evap_feats

,layer_name,layer_description,layer_number
0,pet1,Monthly potential evapotranspiration from the ...,56
1,pet2,Monthly potential evapotranspiration from the ...,57
2,pet3,Monthly potential evapotranspiration from the ...,58
3,pet4,Monthly potential evapotranspiration from the ...,59
4,pet5,Monthly potential evapotranspiration from the ...,60
5,pet6,Monthly potential evapotranspiration from the ...,61
6,pet7,Monthly potential evapotranspiration from the ...,62
7,pet8,Monthly potential evapotranspiration from the ...,63
8,pet9,Monthly potential evapotranspiration from the ...,64
9,pet10,Monthly potential evapotranspiration from the ...,65


In [52]:
biowater_feats = pd.read_json(clim_meta["table_3"])
biowater_feats

,Bioclimatic variable,Description
0,cwd,Annual climatic water deficit (mm)
1,ndm,Number of dry months in the year


In [53]:
biowater_feats.columns = ["layer_name", "layer_description"]

In [54]:
current_start_layer = max(evap_feats["layer_number"]) + 1
biowater_feats["layer_number"] = range(current_start_layer, current_start_layer + len(biowater_feats))
biowater_feats

,layer_name,layer_description,layer_number
0,cwd,Annual climatic water deficit (mm),69
1,ndm,Number of dry months in the year,70


In [55]:
meta_dfs = [bio_monthly_feats, bioclim_feats, evap_feats, biowater_feats]

In [56]:
def meta_merge_clim_df(clim_df: pd.DataFrame, meta_dfs: List[pd.DataFrame])-> pd.DataFrame:
    merge_result_dfs = []
    for meta_df in meta_dfs:
        result_df = pd.merge(clim_df, meta_df, on="layer_number")
        merge_result_dfs.append(result_df)
    
    merged_df = pd.concat(merge_result_dfs).reset_index(drop=True)

    return merged_df

In [57]:
df_clim = meta_merge_clim_df(df_clim, meta_dfs)

In [58]:
df_clim

,layer_number,geoclim_feature,geoclim_type,layer_name,layer_description
0,1,Monthly minimum temperature (°C x 10),clim,tmin1,Monthly minimum temperature (°C x 10) - January
1,2,Monthly minimum temperature (°C x 10),clim,tmin2,Monthly minimum temperature (°C x 10) - February
2,3,Monthly minimum temperature (°C x 10),clim,tmin3,Monthly minimum temperature (°C x 10) - March
3,4,Monthly minimum temperature (°C x 10),clim,tmin4,Monthly minimum temperature (°C x 10) - April
4,5,Monthly minimum temperature (°C x 10),clim,tmin5,Monthly minimum temperature (°C x 10) - May
...,...,...,...,...,...
65,66,Monthly potential evapotranspiration (mm.month-1),clim,pet11,Monthly potential evapotranspiration from the ...
66,67,Monthly potential evapotranspiration (mm.month-1),clim,pet12,Monthly potential evapotranspiration from the ...
67,68,Annual potential evapotranspiration (mm),clim,pet,Annual potential evapotranspiration from the T...
68,69,Annual climatic water deficit (mm),clim,cwd,Annual climatic water deficit (mm)


env

In [59]:
with open(enviro_dir / defs.geoclim_files["env_data_format"], "r") as f:
    env_format = json.load(f)

In [60]:
df_env = pd.read_json(env_format["table_0"])

In [61]:
df_env.columns = ["layer_number", "geoclim_feature"]

In [62]:
df_env["geoclim_type"] = "env"

In [63]:
df_env

,layer_number,geoclim_feature,geoclim_type
0,1,Altitude (m),env
1,2,Slope (in degree),env
2,3,"Aspect (clockwise from North, in degree)",env
3,4,Solar radiation (Wh.m-2.day-1),env
4,5,"Geology (Kew Botanical Garden, 1997)",env
5,6,"Soil (Pelletier, 1981)",env
6,7,"Vegetation (Kew Botanical Garden, 2007)",env
7,8,"Watersheds (Pearson, 2009)",env
8,9,Percentage of forest cover for the year 2010 (%).,env


In [64]:
current_start_layer = max(df_clim["layer_number"]) + 1
df_env["layer_number"] = range(current_start_layer, current_start_layer + len(df_env))

In [65]:
df_env

,layer_number,geoclim_feature,geoclim_type
0,71,Altitude (m),env
1,72,Slope (in degree),env
2,73,"Aspect (clockwise from North, in degree)",env
3,74,Solar radiation (Wh.m-2.day-1),env
4,75,"Geology (Kew Botanical Garden, 1997)",env
5,76,"Soil (Pelletier, 1981)",env
6,77,"Vegetation (Kew Botanical Garden, 2007)",env
7,78,"Watersheds (Pearson, 2009)",env
8,79,Percentage of forest cover for the year 2010 (%).,env


In [66]:
df_env["layer_name"] = df_env["geoclim_feature"].str.split(" ").str[0].str.lower()

In [67]:
df_env.loc[df_env["layer_number"] == 79, "layer_name"] = "forestcover"

In [68]:
df_env["layer_description"] = "unknown"

In [69]:
with open(enviro_dir / defs.geoclim_files["env_metadata"], "r") as f:
    env_meta = json.load(f)

In [70]:
env_meta_str = env_meta["table_0"]
data = json.loads(env_meta_str)


In [71]:
geology_description = {}
for i, val in data["Raster value"].items():
    rock_type = data["Rock type"][i]
    geology_description[val] = rock_type

In [72]:
geology_description

{1: 'Alluvial & Lake deposits',
 2: 'Unconsolidated Sands',
 4: 'Mangrove Swamp',
 5: 'Tertiary Limestones + Marls & Chalks',
 6: 'Sandstones',
 7: 'Mesozoic Limestones + Marls (inc. "Tsingy")',
 9: 'Lavas (including Basalts & Gabbros)',
 10: 'Basement Rocks (Ign & Met)',
 11: 'Ultrabasics',
 12: 'Quartzites',
 13: 'Marble (Cipolin)'}

In [73]:
df_env.loc[df_env["layer_name"] == "geology", "layer_description"] = [geology_description]

In [74]:
df_env

,layer_number,geoclim_feature,geoclim_type,layer_name,layer_description
0,71,Altitude (m),env,altitude,unknown
1,72,Slope (in degree),env,slope,unknown
2,73,"Aspect (clockwise from North, in degree)",env,aspect,unknown
3,74,Solar radiation (Wh.m-2.day-1),env,solar,unknown
4,75,"Geology (Kew Botanical Garden, 1997)",env,geology,"{1: 'Alluvial & Lake deposits', 2: 'Unconsolid..."
5,76,"Soil (Pelletier, 1981)",env,soil,unknown
6,77,"Vegetation (Kew Botanical Garden, 2007)",env,vegetation,unknown
7,78,"Watersheds (Pearson, 2009)",env,watersheds,unknown
8,79,Percentage of forest cover for the year 2010 (%).,env,forestcover,unknown


In [75]:
pd.concat([df_clim, df_env])

,layer_number,geoclim_feature,geoclim_type,layer_name,layer_description
0,1,Monthly minimum temperature (°C x 10),clim,tmin1,Monthly minimum temperature (°C x 10) - January
1,2,Monthly minimum temperature (°C x 10),clim,tmin2,Monthly minimum temperature (°C x 10) - February
2,3,Monthly minimum temperature (°C x 10),clim,tmin3,Monthly minimum temperature (°C x 10) - March
3,4,Monthly minimum temperature (°C x 10),clim,tmin4,Monthly minimum temperature (°C x 10) - April
4,5,Monthly minimum temperature (°C x 10),clim,tmin5,Monthly minimum temperature (°C x 10) - May
...,...,...,...,...,...
4,75,"Geology (Kew Botanical Garden, 1997)",env,geology,"{1: 'Alluvial & Lake deposits', 2: 'Unconsolid..."
5,76,"Soil (Pelletier, 1981)",env,soil,unknown
6,77,"Vegetation (Kew Botanical Garden, 2007)",env,vegetation,unknown
7,78,"Watersheds (Pearson, 2009)",env,watersheds,unknown
